In [ ]:
# SETUP

# Install our additional project requirements
%pip install --quiet --no-index --find-links=/kaggle/input/mammography-move/pip -Ur /kaggle/input/mammography-move/kaggle/mammography/requirements/main.in
%env PYTHONPATH /kaggle/input/mammography-move/kaggle/

In [ ]:
# hack pythonpath
import sys

sys.path = ["/kaggle/input/mammography-move/kaggle/"]

In [ ]:
import pandas as pd
from mammography.src.prep_meta import get_breast_metadata

meta = pd.read_csv("/kaggle/input/rsna-breast-cancer-detection/test.csv")
breasts = get_breast_metadata(meta)
breasts.to_pickle("/kaggle/working/png/val.pickle")

In [ ]:
# Generate PNGs
!python -m mammography.src.preprocess_images \
    /kaggle/input/rsna-breast-cancer-detection/test_images/ \
    /kaggle/working/png/val.pickle \
    /kaggle/working/png

In [ ]:
# Submit
!python -m mammography.src.submit \
    +trainer.accelerator=gpu \
    datamodule.checkpoint_path=/kaggle/input/mammography-move/checkpoint.ckpt \
    datamodule.data_dir=/kaggle/working/png \
    model.checkpoint_path=/kaggle/input/mammography-move/checkpoint.ckpt